In [1]:
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
TRI_DATA_DIR = './trip_data/'
CLI_DATA_DIR = './climate_data/'

### Trip Data Loading

In [3]:
df_trips = pd.read_csv(TRI_DATA_DIR+'2019_wTime.csv')

In [4]:
df_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,year,month,day,hour,weekend,periods,holiday
0,4581278,199751,1547.0,7021.0,01/01/2019 00:08,Bay St / Albert St,7233.0,01/01/2019 00:33,King / Cowan Ave - SMART,1296,Annual Member,2019,1,1,0,0,3,1.0
1,4581279,294730,1112.0,7160.0,01/01/2019 00:10,King St W / Tecumseth St,7051.0,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2947,Annual Member,2019,1,1,0,0,3,1.0
2,4581280,197252,589.0,7055.0,01/01/2019 00:15,Jarvis St / Carlton St,7013.0,01/01/2019 00:25,Scott St / The Esplanade,2293,Annual Member,2019,1,1,0,0,3,1.0
3,4581281,171700,259.0,7012.0,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235.0,01/01/2019 00:20,Bay St / College St (West Side) - SMART,283,Annual Member,2019,1,1,0,0,3,1.0
4,4581282,306314,281.0,7041.0,01/01/2019 00:19,Edward St / Yonge St,7257.0,01/01/2019 00:24,Dundas St W / St. Patrick St,1799,Annual Member,2019,1,1,0,0,3,1.0


In [5]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414152 entries, 0 to 2414151
Data columns (total 18 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Trip Id             int64  
 1   Subscription Id     int64  
 2   Trip  Duration      float64
 3   Start Station Id    float64
 4   Start Time          object 
 5   Start Station Name  object 
 6   End Station Id      float64
 7   End Time            object 
 8   End Station Name    object 
 9   Bike Id             int64  
 10  User Type           object 
 11  year                int64  
 12  month               int64  
 13  day                 int64  
 14  hour                int64  
 15  weekend             int64  
 16  periods             int64  
 17  holiday             float64
dtypes: float64(4), int64(9), object(5)
memory usage: 331.5+ MB


## All Time 

In [6]:
df_stations = df_trips[['Start Station Id', 'Start Station Name']].drop_duplicates()
df_stations['key'] = 0

In [7]:
# drop na
df_stations.dropna(inplace=True)

In [8]:
df_date = pd.read_csv(CLI_DATA_DIR+'climate.csv')[['Year', 'Month', 'Day', 'Time']].drop_duplicates()
df_date['key'] = 0

In [9]:
def weekend(m, d):
    date = datetime.strptime('2019-%02d-%02d' % (m, d), '%Y-%m-%d').weekday()
    if date >= 5:
        return 1
    else:
        return 0

In [10]:
# get weekend
df_date['Weekend'] = [weekend(x[1]['Month'], x[1]['Day']) for x in df_date.iterrows()]
df_date.head()

,Year,Month,Day,Time,key,Weekend
0,2019,1,1,00:00,0,0
1,2019,1,1,01:00,0,0
2,2019,1,1,02:00,0,0
3,2019,1,1,03:00,0,0
4,2019,1,1,04:00,0,0


In [11]:
# holiday
df_date['Holiday'] = 0
# New Year's Day
df_date.loc[(df_date['Month'] == 1) & (df_date['Day'] == 1), 'Holiday'] = 1
# Family Day
df_date.loc[(df_date['Month'] == 2) & (df_date['Day'] == 18), 'Holiday'] = 1
# Good Friday
df_date.loc[(df_date['Month'] == 4) & (df_date['Day'] == 19), 'Holiday'] = 1
# Victoria Day
df_date.loc[(df_date['Month'] == 5) & (df_date['Day'] == 20), 'Holiday'] = 1
# Canada Day
df_date.loc[(df_date['Month'] == 7) & (df_date['Day'] == 1), 'Holiday'] = 1
# Labour Day
df_date.loc[(df_date['Month'] == 9) & (df_date['Day'] == 2), 'Holiday'] = 1
# Thanksgiving Day
df_date.loc[(df_date['Month'] == 10) & (df_date['Day'] == 14), 'Holiday'] = 1
# Christmas Day
df_date.loc[(df_date['Month'] == 12) & (df_date['Day'] == 25), 'Holiday'] = 1
# Boxing Day
df_date.loc[(df_date['Month'] == 12) & (df_date['Day'] == 26), 'Holiday'] = 1

In [12]:
df = pd.merge(df_stations, df_date, how='left', on='key')
df.drop(columns=['key'], inplace=True)

In [13]:
df.rename(columns={'Start Station Id':'Station Id', 'Start Station Name':'Station Name'}, inplace=True)

In [14]:
df.head()

,Station Id,Station Name,Year,Month,Day,Time,Weekend,Holiday
0,7021.0,Bay St / Albert St,2019,1,1,00:00,0,1
1,7021.0,Bay St / Albert St,2019,1,1,01:00,0,1
2,7021.0,Bay St / Albert St,2019,1,1,02:00,0,1
3,7021.0,Bay St / Albert St,2019,1,1,03:00,0,1
4,7021.0,Bay St / Albert St,2019,1,1,04:00,0,1


## Checkin per Hour 

In [15]:
# drop na
df_trips = df_trips[df_trips['End Station Id'].notna()]

In [16]:
def date2cols(date):
    new_date = datetime.strptime(date, '%m/%d/%Y %H:%M')
    new_date = new_date.strftime('%Y %m %d %H').strip().split(' ')
    return [int(new_date[0]), int(new_date[1]), int(new_date[2]), int(new_date[3])]

In [17]:
df_trips = df_trips.loc[:,:'End Station Name']
# add the month, day and hour columns
end_time = df_trips['End Time'].values
time_cols = [date2cols(t) for t in end_time]
time_cols = pd.DataFrame(time_cols, columns = ['Year', 'Month', 'Day', 'Hour'], index = df_trips.index)
df_trips = pd.concat([df_trips, time_cols], axis = 1)
df_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Year,Month,Day,Hour
0,4581278,199751,1547.0,7021.0,01/01/2019 00:08,Bay St / Albert St,7233.0,01/01/2019 00:33,King / Cowan Ave - SMART,2019,1,1,0
1,4581279,294730,1112.0,7160.0,01/01/2019 00:10,King St W / Tecumseth St,7051.0,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2019,1,1,0
2,4581280,197252,589.0,7055.0,01/01/2019 00:15,Jarvis St / Carlton St,7013.0,01/01/2019 00:25,Scott St / The Esplanade,2019,1,1,0
3,4581281,171700,259.0,7012.0,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235.0,01/01/2019 00:20,Bay St / College St (West Side) - SMART,2019,1,1,0
4,4581282,306314,281.0,7041.0,01/01/2019 00:19,Edward St / Yonge St,7257.0,01/01/2019 00:24,Dundas St W / St. Patrick St,2019,1,1,0


In [18]:
checkin = df_trips.groupby(by=['End Station Id', 'End Station Name', 'Year', 'Month', 'Day', 'Hour'], dropna=False). \
          size().reset_index(name='Checkin per Hour')

In [19]:
checkin.rename(columns={'End Station Id':'Station Id', 'End Station Name':'Station Name'}, inplace=True)

In [20]:
# change format
checkin['Hour'] = checkin['Hour'].apply(lambda x: '{:0>2d}:00'.format(int(x)))
checkin.rename(columns={'Hour':'Time'}, inplace=True)

In [21]:
checkin.head()

,Station Id,Station Name,Year,Month,Day,Time,Checkin per Hour
0,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,01:00,1
1,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,02:00,2
2,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,23:00,1
3,7000.0,Fort York Blvd / Capreol Ct,2019,1,2,11:00,3
4,7000.0,Fort York Blvd / Capreol Ct,2019,1,2,12:00,1


## Checkout per Hour

In [22]:
# drop na
df_trips = df_trips[df_trips['Start Station Id'].notna()]

In [23]:
df_trips = df_trips.loc[:,:'End Station Name']
# add the month, day and hour columns
start_time = df_trips['Start Time'].values
time_cols = [date2cols(t) for t in start_time ]
time_cols = pd.DataFrame(time_cols, columns = ['Year', 'Month', 'Day', 'Hour'], index = df_trips.index)
df_trips = pd.concat([df_trips, time_cols], axis = 1)
df_trips.head()

,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Year,Month,Day,Hour
0,4581278,199751,1547.0,7021.0,01/01/2019 00:08,Bay St / Albert St,7233.0,01/01/2019 00:33,King / Cowan Ave - SMART,2019,1,1,0
1,4581279,294730,1112.0,7160.0,01/01/2019 00:10,King St W / Tecumseth St,7051.0,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2019,1,1,0
2,4581280,197252,589.0,7055.0,01/01/2019 00:15,Jarvis St / Carlton St,7013.0,01/01/2019 00:25,Scott St / The Esplanade,2019,1,1,0
3,4581281,171700,259.0,7012.0,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235.0,01/01/2019 00:20,Bay St / College St (West Side) - SMART,2019,1,1,0
4,4581282,306314,281.0,7041.0,01/01/2019 00:19,Edward St / Yonge St,7257.0,01/01/2019 00:24,Dundas St W / St. Patrick St,2019,1,1,0


In [24]:
checkout = df_trips.groupby(by=['Start Station Id', 'Start Station Name', 'Year', 'Month', 'Day', 'Hour'], dropna=False). \
          size().reset_index(name='Checkout per Hour')

In [25]:
checkout.rename(columns={'Start Station Id':'Station Id', 'Start Station Name':'Station Name'}, inplace=True)

In [26]:
# change format
checkout['Hour'] = checkout['Hour'].apply(lambda x: '{:0>2d}:00'.format(int(x)))
checkout.rename(columns={'Hour':'Time'}, inplace=True)

In [27]:
checkout.head()

,Station Id,Station Name,Year,Month,Day,Time,Checkout per Hour
0,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,07:00,2
1,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,11:00,3
2,7000.0,Fort York Blvd / Capreol Ct,2019,1,1,15:00,4
3,7000.0,Fort York Blvd / Capreol Ct,2019,1,2,07:00,1
4,7000.0,Fort York Blvd / Capreol Ct,2019,1,2,08:00,5


## Outer Join 

In [28]:
# join data
df = pd.merge(df, checkin,  how='left', 
              left_on=['Station Id', 'Station Name', 'Year', 'Month', 'Day', 'Time'], 
              right_on=['Station Id', 'Station Name', 'Year', 'Month', 'Day', 'Time'])
df = pd.merge(df, checkout,  how='left', 
              left_on=['Station Id', 'Station Name', 'Year', 'Month', 'Day', 'Time'], 
              right_on=['Station Id', 'Station Name', 'Year', 'Month', 'Day', 'Time'])

In [29]:
# fill checkin/checkout nan
df.fillna(0, inplace=True)

In [30]:
df.loc[:, 'Station Id'] = df['Station Id'].apply(int)

In [31]:
df.head()

,Station Id,Station Name,Year,Month,Day,Time,Weekend,Holiday,Checkin per Hour,Checkout per Hour
0,7021,Bay St / Albert St,2019,1,1,00:00,0,1,0.0,1.0
1,7021,Bay St / Albert St,2019,1,1,01:00,0,1,0.0,0.0
2,7021,Bay St / Albert St,2019,1,1,02:00,0,1,0.0,0.0
3,7021,Bay St / Albert St,2019,1,1,03:00,0,1,0.0,0.0
4,7021,Bay St / Albert St,2019,1,1,04:00,0,1,0.0,0.0


## Weekend and Holiday

In [32]:
df.drop(df[df['Year'] == 2020].index, inplace = True) 

## Save to CSV 

In [33]:
df.to_csv(TRI_DATA_DIR+'stations_check.csv', index=False)

In [34]:
df.groupby('Station Id').sum()[['Checkin per Hour', 'Checkout per Hour']]

,Checkin per Hour,Checkout per Hour
Station Id,,
7000,14481.0,15246.0
7001,9080.0,7726.0
7002,11472.0,12027.0
7003,5010.0,5426.0
7004,5131.0,5382.0
...,...,...
7475,2718.0,2640.0
7489,3226.0,3090.0
7491,4504.0,4090.0
